In [9]:
import pickle
from sc2.game_info import GameInfo

import numpy as np
from scipy import ndimage
import plotly.express as px
import plotly.graph_objects as go

In [10]:
import sys
sys.path.append("../bot")

from extensions.poisson_multigrid import mg_opt, convolve_jacobi

In [11]:
with open("../resources/Site Delta AIE.pkl", "rb") as f:
    game_info: GameInfo = pickle.load(f)

In [12]:
b = game_info.pathing_grid.data_numpy.T
b *= (convolve_jacobi(b) > 0)
px.imshow(b)

In [13]:
f = game_info.terrain_height.data_numpy.T
f = np.where(f == np.max(f), 100.0, 0.0)
f += 100 * np.random.rand(*f.shape)
f *= b
f += 1
f[:f.shape[0]//2,:] *= -1.0
#f -= f.mean()
#f /= np.max(abs(f))
#f /= np.prod(f.shape)
px.imshow(f)

In [14]:
x = f.copy()
for _ in range(100):
    x = mg_opt(x, np.zeros_like(x), b)
px.imshow(np.where(b, x, np.nan))

In [15]:
x = np.zeros_like(b)
for i in range(100):
    x = mg_opt(x, f, b)
px.imshow(np.where(b, x, np.nan))

In [16]:
g = b[:-1,:-1,np.newaxis] * np.dstack((
    np.diff(x, axis=0)[:,:-1],
    np.diff(x, axis=1)[:-1,:],
    np.zeros_like(x)[:-1,:-1],
))
g /= 1e-10 + np.linalg.norm(g, axis=2, keepdims=True)
px.imshow(g, zmin=np.min(g), zmax=np.max(g))